<h1><center>Final Assignement</center></h1>

<a class="anchor" id="0"></a>
### Table of content
1. [Loading in and reading data](#1)<br>
2. [Feature Engineering](#2)<br>
3. [Loading in and reading data](#3)<br>

This is the first Notebook. It's used to retrieve the data from the M5 challenge, merges dataframes, does feature engineering and then writes the output to pickle files which can then be used in the next script as a starting point. Questions 1-3 of the methodology of Deep Learning with Python ("Define the Problem", "Choosing a measure of success", "Deciding on an evaluation protocol") are answered in the report itself. I will start with step 4: "Preparing your data" from here.

In [1]:
#Import data handling libraries
import pandas as pd
import numpy as np
import requests
import helper_functions as hf

In [2]:
# Set variables
NUM_ITEMS = 30490

#Specify directories
CODE_ENV = hf.detect_environment()
dirs = hf.get_paths(CODE_ENV)
parent_dir = dirs['parent_dir']
res_dir = dirs['res_dir']
src_dir = dirs['src_dir']
prc_dir = dirs['prc_dir']

Environment: local


<br>
<a id='1'></a>
<h2>1. Loading data from CSV files, reduce memory usage, convert and merge with other dataframes</h2>

In [ ]:
# Import the provided csv files
df_cal = pd.read_csv(res_dir + 'calendar.csv')
df_prices = pd.read_csv(res_dir + 'sell_prices.csv')
df_train_eval = pd.read_csv(res_dir + 'sales_train_evaluation.csv')

In [ ]:
# Inspect dataframes
# print('\nCalendar dataframe: ')
# print(df_cal.head())
# print('\nPrices dataframe: ')
# print(df_prices.head())
# print('\nTrain evaluation dataframe: ')
# print(df_train_eval.head())

In [ ]:
# Helper function to reduce memory usage of dataframes
def reduce_df_mem_usage(df, df_name):
    """ 
    Helper function to iterate all columns of given dataframe and check and set for smallest dtype to reduce memory usage
    Taken and adapted from the widely used function which is available for instance here: 
    https://www.kaggle.com/code/gemartin/load-data-reduce-memory-usage/notebook
    There is a flaw in the integer section in the publicly available function which is that it possibly introduces rounding errors when converting to a smaller dtype
    """

    #Print original memory usage
    print('Dataframe ' + df_name + ' is being processed...')
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    #Iterate through each column
    for col in df.columns:
        #Get dtype
        col_type = df[col].dtype
        
        #try except throw block to try if the following code works, otherwise skip this loop
        try:

            #If type is not an object, therefore numerical, get biggest and smallest values
            if col_type != object:
                c_min = df[col].min()
                c_max = df[col].max()
                
                #If type is int
                if str(col_type)[:3] == 'int':
                    #If min value is greater than min value of given dtype and max value is smaller than max value of given dtype -> adjust dtype
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                #Downcasting of float values leads to rounding errors -> as precision is crucial for float values, no downcasting is performed

            #If none of the above, then assumption that  finite set of possible values -> convert to category which is internally stored as int but when queried returns the string
            else:
                df[col] = df[col].astype('category')
        
        except:
            pass

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    print('---------------------------------------------------\n')

    return df

In [ ]:
# Read in all data used later on
df_prices = reduce_df_mem_usage(df_prices, 'df_prices')
df_cal = reduce_df_mem_usage(df_cal, 'df_cal')
df_train_eval = reduce_df_mem_usage(df_train_eval, 'df_train_eval')

The final submission dataframe should also contain the validation data. The df_train_eval already contains this information so it will be extracted and transformed into the suitable output format. It will then be written out to a pkl file and read in and concatenated with the predictions from the model in the next notebook

In [ ]:
# get column id and all columns from d_1914 to d_1941
df_val = df_train_eval.iloc[:, [0] + list(range(-28, 0))]
# rename day columns to F_1 to F_28
# Generate the new column names for columns 1 to 29
new_column_names = {df_val.columns[i]: 'F' + str(i) for i in range(1, 29)}

# Rename the columns in the DataFrame
df_val.rename(columns=new_column_names, inplace=True)

# Replace string evaluation with validation in the id column
df_val['id'] = df_val['id'].str.replace('evaluation', 'validation')

# write out
df_val.to_pickle(src_dir + 'processed_data/' + 'df_1.pkl')

# delete
del df_val

In the following I will create 3 dataframes that will be stored in pickle format so that unused dataframes can be deleted from memory and the information is clearly separated.
The dataframes are:
- 1. df_1: base grid with the main train information that contains the conversion of the sales data from wide to long format and merge of the sales price
- 2. df_2: Feature engineering
- 3. df_3: contains calendar data and some calendar related generated date features

First convert from wide to long format of the training dataframe. Right now all sales days are in columns. We want them in rows for processing in ML / DL models.

In [ ]:
# Convert from wide to long format
df_conv = pd.melt(df_train_eval,
                    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                    var_name='d', 
                    value_name='sales_amount')

# delete df_train_eval from memory because no longer needed
del df_train_eval 

# Reduce memory usage of df_conv
df_conv = reduce_df_mem_usage(df_conv, 'df_conv')

In [ ]:
# create a dataframe for the next 28 days such that a slice of all 30490 items for day 1 will be copied, the columns remain the same and only for day a new value with d_1942 - d_1969 will be generated with a NaN value for sales_amount
df_future_base = df_conv[df_conv['d']=='d_1'].copy()
# set sales_amount to NaN
df_future_base['sales_amount'] = np.nan
#create an empty dataframe that will hold the future 28 day values
df_future = pd.DataFrame()
#loop 28 times to concat 28 days
for i in range(28):
    df_future_base['d'] = 'd_' + str(1942 + i)
    df_future = pd.concat([df_future, df_future_base.copy()], axis=0)

In [ ]:
# Now concat the future 28 days to the original dataframe
df_conv = pd.concat([df_conv, df_future], axis=0)
df_conv.reset_index(drop=True, inplace=True)

Now we have the issue, that 0 values in the sales_amount column could mean that there were no sales, it could also mean that the sales start of the product was later and the product was just not available.
The sales start date can be infered from the price df. If there is no sales week for a product then that most likely means it wasn't available for sale.
I will first try to delete all rows from df_conv where there were no weeks available for from the price df. This will be done by adding the week information from prices through a left join. Then the week information is available to make a right join on the prices table and thereby filter out all rows where there was no sales week available (the product didn't exist yet).

In [ ]:
####### Cited from the 1st solution (exact citing) #######
## Merging by concat to not lose dtypes
def merge_by_concat_left(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [ ]:
# Combine the converted df with the calendar df
df_conv = merge_by_concat_left(df_conv, df_cal[['d', 'wm_yr_wk']], ['d'])

In [ ]:
# Check output of operation
df_conv.head()

Merge with the prices df to get the sales start date for each product. Then filter out all rows where there was no price available (the product didn't exist yet).

In [ ]:
# Combine the converted df with the prices df
df_conv = merge_by_concat_left(df_conv, df_prices, ['store_id', 'item_id', 'wm_yr_wk'])

In [ ]:
# Create a new column is_available where when column sell_price is NaN the value is 0 (indicates item was not available) and otherwiese 1 (indicates the item was available that day)
df_conv['is_available'] = np.where(df_conv['sell_price'].isna(), 0, 1).astype(np.int8)

# Also set column sales_amount to NaN when sell_price is NaN
df_conv['sell_price'].fillna(0, inplace=True)

In [ ]:
# Check output of operation
df_conv.head()

<br>
<h2>2. Feature engineering</h2>
<a id='2'></a>
Feature engineering will be done here, and then can in the next step be merged with the df_conv dataframe and deleted from memory
- 1 days lag #float 64
- moving average for 7 and 28 days #float 64
- consecutive_zero_sales days
- zero_sales_available

More possible:
- is there a price reduction?
- is there a price increase?
- adjust for inflation?
- consumer sentiment
- holiday
- weather

In [ ]:
# Perform feature engineering
def feature_engineering(df_conv): 
    ################## lag 1 day sales amount ##############################################################################
    df_conv['sales_amount_lag_1'] = df_conv['sales_amount'].shift(NUM_ITEMS)

    # Now the first value of every entry in the sales_amount_lag_1 column is NaN, because there is no value for the first day of the time series, so we will replace these with the mode (most frequent value) of the sales_amount column
    mode_sales_amount = df_conv.groupby('id')['sales_amount'].agg(lambda x: x.mode()[0])

    # Replace the first day's sales_amount_lag_1 for each item with the mode_sales_amount value
    df_conv.loc[df_conv['d'] == 'd_1', 'sales_amount_lag_1'] = df_conv.loc[df_conv['d'] == 'd_1', 'id'].map(mode_sales_amount)
    ########################################################################################################################

    ################## moving average 7 and 28 days #########################
    df_conv['sales_amount_moving_avg_7'] = df_conv.groupby('id')['sales_amount'].transform(lambda x: x.rolling(window=7).mean())
    df_conv['sales_amount_moving_avg_7'] = df_conv['sales_amount_moving_avg_7'].fillna(method='bfill')

    df_conv['sales_amount_moving_avg_28'] = df_conv.groupby('id')['sales_amount'].transform(lambda x: x.rolling(window=28).mean())
    df_conv['sales_amount_moving_avg_28'] = df_conv['sales_amount_moving_avg_28'].fillna(method='bfill')
    #########################################################################

    ################## days consecutive zero sales and if an entry means that this is a zero sale  #########################
    # Step 1: Mark zero sales days where item is available
    df_conv['zero_sales_available'] = np.where((df_conv['sales_amount'] == 0) & (df_conv['is_available'] == 1), 1, 0).astype(np.int8)

    # Function to apply to each group
    def calculate_consecutive_zeros(group):
        # Step 2: Identify change points to reset the count for consecutive zeros
        group['block'] = (group['zero_sales_available'] == 0).cumsum().astype(np.int16)
        
        # Step 3: Count consecutive zeros within each block
        group['consecutive_zero_sales'] = group.groupby('block').cumcount()
        
        # Reset count where 'zero_sales_available' is 0, as these are not zero sales days or the item is not available
        group['consecutive_zero_sales'] = np.where(group['zero_sales_available'] == 1, group['consecutive_zero_sales'], 0).astype(np.int16)
        
        return group

    # Apply the function to each item group
    df_conv = df_conv.groupby('id', group_keys=False).apply(calculate_consecutive_zeros)

    # Drop the 'block' column because no longer needed
    del df_conv['block']

    return df_conv
########################################################################################################################

In [ ]:
df_conv = feature_engineering(df_conv)

In [ ]:
# delete df_prices from memory because no longer needed
del df_prices
del df_conv['wm_yr_wk']
df_conv.reset_index(drop=True, inplace=True)

In [ ]:
# Base steps are done, now lets save to pickle file
df_conv.to_pickle(src_dir + 'processed_data/' + 'df_2.pkl')
del df_conv

<br>
<h2>3. Create the third dataframe with the calendar information</h2>
<a id='2'></a>
Now lets get the calendar information and do some date feature engineering.

In [ ]:
# read df_conv from pickle file 
df_conv = pd.read_pickle(src_dir + 'processed_data/' + 'df_2.pkl')

In [ ]:
# Retrieve only identifying columns of df_conv to keep df small and to merge with df_conv
df_conv = df_conv[['id', 'd']]

In [ ]:
# Pick all useful columns for merge with df_conv
icols = ['date',
         'd',
         'wday',
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']

# Now merge the df_conv with useful features from the prices df
df_conv = merge_by_concat_left(df_conv, df_cal[icols], ['d'])

In [ ]:
# Convert the date column to datetime format
df_conv['date'] = pd.to_datetime(df_conv['date'])

# Do feature engineering on the df_conv dataframe; more features are possible
df_conv['week']  = df_conv['date'].dt.isocalendar().week.astype(np.int8)
df_conv['month'] = df_conv['date'].dt.month.astype(np.int8)
df_conv['year']  = df_conv['date'].dt.year.astype(np.int16)
df_conv['weekend'] = (df_conv['wday']<=2).astype(np.int8) # check with binary for weekend

df_conv['day_month'] = df_conv['date'].dt.day.astype(np.int8)
df_conv['week_month'] = np.ceil(df_conv['day_month'] / 7).astype(np.int8) #check which week of the month, tried vectorized approach
df_conv['max_week_month'] = np.ceil(df_conv['date'].dt.days_in_month / 7).astype(np.int8) # Calculate the maximum week number for each row/date
df_conv['week_month_norm'] = (df_conv['week_month'] - 1) / (df_conv['max_week_month'] - 1).astype(np.float16) # Normalize week_month
df_conv.drop(['max_week_month', 'day_month', 'week_month'], axis=1, inplace=True) # delete columns max_week_month, day_month and week_month because no longer needed

# convert all days strings to integers to reduce memory usage
df_conv['d'] = df_conv['d'].str.replace('d_', '').astype(np.int16)

In [ ]:
######################################## Convert to cyclical data of date time ########################################
# Convert cyclical features with sin and cos
df_conv['month_sin'] = np.sin(2 * np.pi * df_conv['month']/12).astype(np.float16)
df_conv['month_cos'] = np.cos(2 * np.pi * df_conv['month']/12).astype(np.float16)
df_conv.drop('month', axis=1, inplace=True)

df_conv['wday_sin'] = np.sin(2 * np.pi * df_conv['wday']/7).astype(np.float16)
df_conv['wday_cos'] = np.cos(2 * np.pi * df_conv['wday']/7).astype(np.float16)
df_conv.drop('wday', axis=1, inplace=True)

df_conv['week_sin'] = np.sin(2 * np.pi * df_conv['week']/53).astype(np.float32)
df_conv['week_cos'] = np.cos(2 * np.pi * df_conv['week']/53).astype(np.float32)
df_conv.drop('week', axis=1, inplace=True)

# Normalize day of month
df_conv['total_days_in_month'] = df_conv['date'].dt.days_in_month
df_conv['mday_normalized'] = ((df_conv['date'].dt.day - 1) / (df_conv['total_days_in_month'] - 1)).astype(np.float32)

# Normalized day of the year
df_conv['tm_dy_norm'] = (df_conv['date'].dt.dayofyear / df_conv['date'].dt.is_leap_year.map({True: 366, False: 365})).astype(np.float32)
df_conv.drop(['date', 'total_days_in_month'], axis=1, inplace=True)

# Create a continous, normalized day number
df_conv['day_continuous_normalized'] = (df_conv['d'] / 1969).astype(np.float32)  # 1969 train, val and test days in total

# Normalize linear data like year
min_year = df_conv['year'].min()
max_year = df_conv['year'].max()
df_conv['year_normalized'] = ((df_conv['year'] - min_year) / (max_year - min_year)).astype(np.float16)
df_conv.drop('year', axis=1, inplace=True)
########################################################################################################################

In [ ]:
# save to pickle and delete df_cal from memory because no longer needed
df_conv.to_pickle(src_dir + 'processed_data/' + 'df_3.pkl')
del df_cal
del df_conv

In [ ]:
# Now the date column with the old format can be deleted from the df_conv dataframe
df_conv = pd.read_pickle(src_dir + 'processed_data/' + 'df_2.pkl')
del df_conv['d']
del df_conv['id']
df_conv.to_pickle(src_dir + 'processed_data/' + 'df_2.pkl')